In [1]:
import saspy
sas = saspy.SASsession(cfgname='default')

SAS Connection established. Subprocess id is 1804



In [2]:
cars = sas.sasdata('cars', 'sashelp')
cars.head()

,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,36945,33337,3.5,6,265,17,23,4451,106,189
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820,21761,2.0,4,200,24,31,2778,101,172
2,Acura,TSX 4dr,Sedan,Asia,Front,26990,24647,2.4,4,200,22,29,3230,105,183
3,Acura,TL 4dr,Sedan,Asia,Front,33195,30299,3.2,6,270,20,28,3575,108,186
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755,39014,3.5,6,225,18,24,3880,115,197


## Basic usage

In [4]:
# define columns to use as classes or computational vars
by_origin, by_type     = cars.tabulate.classes('origin', 'type')
horsepower, cylinders  = cars.tabulate.vars('horsepower', 'cylinders')

# grab statistics of interest
mean, n                = cars.tabulate.stats('mean', 'n')

# compose these elements into a table
cars.tabulate.table(
    left = by_origin * by_type,
    top  = (horsepower | cylinders) * (mean | n)
)

### Adding options

In [5]:
# assign labels, formats
by_origin, by_type     = cars.tabulate.classes('origin', 'type', labels=False)
horsepower, cylinders  = cars.tabulate.vars('horsepower', 'cylinders')
mean, n                = cars.tabulate.stats('mean', 'n', formats=['6.2', '3.'], labels=['Average', 'Count'])

# you can override or add options in composition by using .with_()
cars.tabulate.table(
    left = by_origin.with_(all='Total') * by_type,
    top  = (horsepower.with_(label='Power!') | cylinders) * (mean.with_(format='6.2') | n)
)

### Alternative: Create class, var, and stat elements individually

In [14]:
by_origin   = cars.tabulate.as_class('origin', label='', all='All Origins')
by_type     = cars.tabulate.as_class('type', label='')

horsepower  = cars.tabulate.as_var('horsepower')
average     = cars.tabulate.stat('mean', format='8.2', label='Mean HP')
n           = cars.tabulate.stat('n', label='Count', format='6.')


cars.tabulate.table(
    left = by_origin,
    top  = by_type * (n | horsepower.with_(label='') * average)
)

## Composition and Re-use

In [15]:
# create some elements for reuse
by_origin   = cars.tabulate.as_class('origin', label='')
by_type     = cars.tabulate.as_class('type', label='')

horsepower  = cars.tabulate.as_var('horsepower', label='Horses')
cylinders   = cars.tabulate.as_var('cylinders', label='Cyls.')
enginesize  = cars.tabulate.as_var('enginesize', label='Engine Size')

average     = cars.tabulate.stat('mean', label='Avg', format='8.2')
stdev       = cars.tabulate.stat('std', label='Std. Dev.', format='5.2')
n           = cars.tabulate.stat('n', label='Count', format='6.0')

# create some compositional fragments
by_origin_and_type = by_origin.with_(all='All') * by_type.with_(all='All')
hpstats            = horsepower * (average | stdev | n)
cylstats           = cylinders * (average | stdev | n)
enginestats        = enginesize * (average | stdev)

In [16]:
# draw a table
cars.tabulate.table(
    left = by_origin_and_type,
    top  = hpstats
)

In [17]:
# draw another table
cars.tabulate.table(
    left = by_type,
    top  = cylstats | enginestats
)

In [18]:
# grab another class when needed, draw another table
drivetrain = cars.tabulate.as_class('drivetrain', label='Drive Train')

cars.tabulate.table(
    left = by_type * drivetrain.with_(all='All'),
    top  = cylstats | enginestats
)

### View the generated code

In [19]:
sas.teach_me_SAS(True)

cars.tabulate.table(
    left = by_type * drivetrain.with_(all='All'),
    top  = cylstats | enginestats
)

proc tabulate data=sashelp.cars ;
  class drivetrain type;
  var enginesize cylinders;
  table type='' * (drivetrain='Drive Train' ALL='All'), (cylinders='Cyls.' * (mean='Avg'*f=8.2 std='Std. Dev.'*f=5.2 n='Count'*f=6.0) enginesize='Engine Size' * (mean='Avg'*f=8.2 std='Std. Dev.'*f=5.2));
run;
